# <font color="blue">1.0 - Cash Flow - Realty Mole Rental API<font>

### <font color="blue">API provided by the source sited below.<font>
---
### <font color="red">*** Code will not work in this notebook unless API Key is updated to an active one. ***<>font

### <font color="red">Also a special thank you goes out to my instructors Ben Mathis and Andrew Riddle, for helping me to adapt this API. It wasn't functioning properly in the original downloaded code.<fong>

# Get Zip Code Rental Data by County

## Overview
| Detail Tag            | Information                                                                                        |
|-----------------------|----------------------------------------------------------------------------------------------------|
| Originally Created By | Ariel Herrera arielherrera@analyticsariel.com |

## History
| Date         | Developed By  | Reason                                                |
|--------------|---------------|-------------------------------------------------------|
| 22nd Jun 2022 | Ariel Herrera | Create notebook. |



---

## <font color="blue">Imports<font>
---

In [9]:
import pandas as pd
from scipy.stats import zscore
import requests
import warnings
from datetime import datetime
from dateutil.relativedelta import relativedelta
import time

warnings.filterwarnings('ignore')

## <font color="blue">Variables</font>
---

In [10]:
#API key provided by RapidAPI.com
rapid_api_key = '6377723b34msh077761c3d376267p15c969jsn8979a9749ad3'

#These are the 41 of the 58 zip codes we will be using for our API pull
zips = ['Los Angeles', 'San Joaquin', 'Sonoma', 'San Francisco', 'San Diego', 'Orange', 'Alameda', 'San Bernardino', 
'San Mateo', 'Kern', 'Shasta', 'Fresno', 'Solano', 'Santa Cruz', 'Contra Costa','San Benito', 'Ventura', 'Nevada', 
'Riverside', 'Madera', 'El Dorado', 'Kings', 'Lake', 'Monterey', 'Imperial','Santa Barbara', 'Tulare', 'Tehama', 
'Napa', 'Placer', 'Marin', 'Santa Clara', 'Sacramento', 'Yuba', 'San Luis Obispo', 'Butte','Mendocino', 'Stanislaus', 
'Sutter', 'Merced', 'Yolo']

## <font color="blue">Functions</font>
---


##### <font color="blue">Using the api key this function gets all the applicable zip codes for the county input. It also checks validity of the zip<font>

In [11]:
def get_rental_market_data(rapid_api_key, zip_code):

  if len(str(zip_code)) != 5:
    return "Invalid Zip Code"

  url = f"https://realty-mole-property-api.p.rapidapi.com/zipCodes/{zip_code}"

  headers = {
    "X-RapidAPI-Key": rapid_api_key,
    "X-RapidAPI-Host": "realty-mole-property-api.p.rapidapi.com"
  }

  return requests.request("GET", url, headers=headers)


##### <font color="blue">This function gets all the activing listings and returns average rent, minimum rent, and maximum rent. For the purposes of this project we are only using "get_latest_rental_data"<font>

In [12]:
def get_latest_rental_data(response):
  # transform data to pandas dataframe
  df = pd.json_normalize(data=response.json())
  # relevant cols
  rent_cols = ['id', 'rentalData.averageRent', 'rentalData.minRent', 'rentalData.maxRent', 'rentalData.totalRentals', 'rentalData.detailed']
  df = df[rent_cols]
  return df.rename(columns={'id': 'zip_code'})

def get_historical_rental_data(response):
  # transform data to pandas dataframe
  df_rent = pd.json_normalize(data=response.json())

  """ get historical data to single dataframe """
  df_list = [] # create empty list
  # get all "detailed" columns
  rent_detail_hist_cols = [x for x in df_rent.columns if 'detailed' and 'history' in x]
  # iterate through "detailed" columns
  for x in rent_detail_hist_cols:
    # get column date
    date_str = x.split('.')[2]
    # get column name
    detail_col = 'rentalData.history.' + date_str + '.detailed'
    # convert historical data to a dataframe
    _df = pd.DataFrame(df_rent[detail_col].iloc[0])
    # create columns
    _df['date_str'] = date_str
    _df['zip_code'] = zip_code
    # append to list
    df_list.append(_df)

  """ combine """
  # comine sub date dataframes
  df_detail = pd.concat(df_list)
  # move date column to front
  df_detail = move_col_to_front(df=df_detail, col_name='date_str')
  df_detail = move_col_to_front(df=df_detail, col_name='zip_code')
  # add feature for previous year
  df_detail['prev_yr_dt'] = df_detail.apply(lambda x: (datetime.strptime(x['date_str'], '%Y-%m') - relativedelta(years=1)).strftime('%Y-%m'), axis=1)

  """ merge """
  df_detail_prev = df_detail.copy()
  df_detail_prev.columns = [x + '_prev_yr' for x in df_detail_prev.columns]
  # merge
  df_merge = pd.merge(df_detail, df_detail_prev, how='left', left_on=['prev_yr_dt', 'bedrooms'], right_on=['date_str_prev_yr', 'bedrooms_prev_yr']).drop_duplicates()
  # identify outliers
  df_merge['averageRent_zscore'] = df_merge[['averageRent']].apply(zscore)

  """ filter """
  # filter
  df_filter = df_merge.copy()
  df_filter = df_filter.loc[(df_filter['bedrooms'] <= 4) & (df_filter['averageRent_zscore'] <= 2)]
  df_filter = df_filter.drop(columns=['prev_yr_dt', 'date_str_prev_yr', 'prev_yr_dt_prev_yr', 'zip_code_prev_yr'])
  # add features
  df_filter['YoY_avg_rent'] = \
    (df_filter['averageRent'] - df_filter['averageRent_prev_yr']) / df_filter['averageRent_prev_yr']
  df_filter['YoY_total_rentals'] = \
    (df_filter['totalRentals'] - df_filter['totalRentals_prev_yr']) / df_filter['totalRentals_prev_yr']
  return df_filter

### <font color="blue">Multiple Zip Codes - All for a county</font>

##### <font color="blue">This is the specific function for when pulling multiple zip codes.<font>

In [13]:
geo_data_url = 'https://raw.githubusercontent.com/scpike/us-state-county-zip/master/geo-data.csv'
df_geo = pd.read_csv(geo_data_url)
print('Num of rows:', len(df_geo))
print('Num of columns:', len(df_geo.columns))
df_geo.head()

Num of rows: 33103
Num of columns: 6


,state_fips,state,state_abbr,zipcode,county,city
0,1,Alabama,AL,35004,St. Clair,Acmar
1,1,Alabama,AL,35005,Jefferson,Adamsville
2,1,Alabama,AL,35006,Jefferson,Adger
3,1,Alabama,AL,35007,Shelby,Keystone
4,1,Alabama,AL,35010,Tallapoosa,New site


In [14]:

# filter on single county
df_geo_county = df_geo.loc[(df_geo['state_abbr'] == 'CA') &
                           (df_geo['county'] == 'Placer')]


# create feature to check if zip code is valid
df_geo_county['valid_zip_code'] = df_geo_county.apply(lambda x: x['zipcode'].isnumeric(), axis=1)
print('Dataset:')
print(df_geo_county.groupby(['valid_zip_code'])['zipcode'].count())
print(' ')
print('Invalid zip codes:', df_geo_county.loc[df_geo_county['valid_zip_code'] == False]['zipcode'].unique())







Dataset:
valid_zip_code
False     1
True     29
Name: zipcode, dtype: int64
 
Invalid zip codes: ['957XX']


In [15]:
# filter on only valid zip codes
df_geo_county_valid = df_geo_county.loc[df_geo_county['valid_zip_code'] == True]
print('Num of VALID zip codes:', len(df_geo_county_valid))
df_geo_county_valid.head()

Num of VALID zip codes: 29


,state_fips,state,state_abbr,zipcode,county,city,valid_zip_code
3337,6,California,CA,95602,Placer,Auburn,True
3338,6,California,CA,95603,Placer,Auburn,True
3361,6,California,CA,95631,Placer,Foresthill,True
3375,6,California,CA,95648,Placer,Lincoln,True
3376,6,California,CA,95650,Placer,Loomis,True


##### <font color="blue"> Splitting all zips. <font>

In [16]:
_all_zips = {}


for county in zips:
    df_geo_county = df_geo.loc[(df_geo['state_abbr'] == 'CA') &
                     (df_geo['county'] == county)]
    df_geo_county['valid_zip_code'] = df_geo_county.apply(lambda x: x['zipcode'].isnumeric(), axis=1)
    df_geo_county_valid = df_geo_county.loc[df_geo_county['valid_zip_code'] == True]

                                                          
                                                          
    _df_latest_list = {}
    _df_historical_list = []                                                      
# iterate through each zip code in the list
    for zip_code in df_geo_county_valid['zipcode'].tolist():
      print('Getting data for zipcode:', zip_code)

      # for each zip code get rental data
      zip_code=int(zip_code)
      response = get_rental_market_data(rapid_api_key, zip_code)

      try:

        # transform response
        # 1) latest rental data
        _df_latest = get_latest_rental_data(response)
        _df_latest_list[zip_code] = _df_latest.iloc[0,-1]

        # 2) historical rental data
        _df_historical = get_historical_rental_data(response)
        _df_historical_list.append(_df_historical)

      except:

        print('**Error in retrieving data for:', zip_code)

      time.sleep(1) # pause for 2 seconds (max 2 requests per second)
    _all_zips.update(_df_latest_list)

Getting data for zipcode: 90001
**Error in retrieving data for: 90001
Getting data for zipcode: 90002
**Error in retrieving data for: 90002
Getting data for zipcode: 90003
**Error in retrieving data for: 90003
Getting data for zipcode: 90004
**Error in retrieving data for: 90004
Getting data for zipcode: 90005
**Error in retrieving data for: 90005
Getting data for zipcode: 90006
**Error in retrieving data for: 90006
Getting data for zipcode: 90007
**Error in retrieving data for: 90007
Getting data for zipcode: 90008
**Error in retrieving data for: 90008
Getting data for zipcode: 90010
**Error in retrieving data for: 90010
Getting data for zipcode: 90011
**Error in retrieving data for: 90011
Getting data for zipcode: 90012
**Error in retrieving data for: 90012
Getting data for zipcode: 90013
**Error in retrieving data for: 90013
Getting data for zipcode: 90014
**Error in retrieving data for: 90014
Getting data for zipcode: 90015
**Error in retrieving data for: 90015
Getting data for zip

KeyboardInterrupt: 

## <font color="blue">Combining Data and Exporting<fong>
---

##### <font color="blue">This for exporting the data. Created by Ben Mathis.<font>

In [109]:
flattened_zips = {}

for k, v in _all_zips.items():
    new_columns = {}

    bed = 0

    for entry in v:
        if bed > 5 or entry['bedrooms'] > 5:
            
            break

        if entry['bedrooms'] > bed:
            
            bed = entry['bedrooms']

        next_entry = {}

        for k2, v2 in entry.items():
            if k2 != 'bedrooms':
                next_entry[f'bed{bed}_'+k2] = v2

        new_columns.update(next_entry)
       
    flattened_zips[k] = new_columns

In [110]:
fz = pd.DataFrame(flattened_zips)

In [121]:
fz.isna().sum()[fz.isna().sum() > 0]

90001     4
90010     4
90012     4
90013     8
90014     4
         ..
95388    16
95612    20
95627    16
95645    20
95694     4
Length: 734, dtype: int64

In [133]:
#Export
fz.to_csv('data/cleaned/rent.csv')